# Resource Mapping for Material Pegging
## Two-tier lookup: DP Line Utilization -> L2Ph1_Detail

**Logic:**
1. Check if Product_ID exists in DP Line Utilization sheet
2. If not found, search in Aug'25_L2_DP_Plan_Circulation_V2 (L2Ph1_Detail)
3. Map Resource_ID and Resource_Description to each Product_ID
4. Handle hierarchy: Packing -> Assembly -> Filling

In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

print("="*120)
print("RESOURCE MAPPING FOR MATERIAL PEGGING - TWO-TIER LOOKUP")
print("="*120)

snp_file = "/home/supriyo/Downloads/Biocon_nw/ParkourSC_SNP.xlsx"
l2_plan_file = "/home/supriyo/Downloads/Biocon_nw/Aug'25_L2_DP_Plan_Circulation_V2.xlsx"

print(f"SNP File: {snp_file}")
print(f"L2 Plan File: {l2_plan_file}")

In [ ]:
def normalize_text(text):
    if pd.isna(text) or text is None:
        return None
    text = str(text).strip()
    text = re.sub(r'\\s+', ' ', text)
    return text if text else None

def normalize_product_no(value):
    if pd.isna(value) or value is None:
        return None
    text = str(value).strip()
    cleaned = re.sub(r'[^a-zA-Z0-9]', '', text)
    return cleaned if cleaned else None

print("Normalization functions loaded")

In [ ]:
print("\n" + "="*120)
print("TIER 1: LOAD DP LINE UTILIZATION")
print("="*120)

tier1_resource_map = {}

try:
    df_dp_util = pd.read_excel(snp_file, sheet_name="DP Line Utilization", header=None, skiprows=2, nrows=240)
    
    print(f"Sheet shape: {df_dp_util.shape}")
    print(f"Columns available: {df_dp_util.shape[1]}")
    
    for _, row in df_dp_util.iterrows():
        product_id = normalize_product_no(row.iloc[4]) if len(row) > 4 else None
        resource_id = normalize_text(row.iloc[1]) if len(row) > 1 else None
        resource_desc = normalize_text(row.iloc[2]) if len(row) > 2 else None
        product_desc = normalize_text(row.iloc[5]) if len(row) > 5 else None
        molecule = normalize_text(row.iloc[3]) if len(row) > 3 else None
        
        if product_id and product_id != '0':
            tier1_resource_map[product_id] = {
                'Resource_ID': resource_id,
                'Resource_Description': resource_desc,
                'Product_Description': product_desc,
                'Molecule': molecule,
                'Source': 'DP_Line_Utilization'
            }
    
    print(f"Tier 1 Resource Map: {len(tier1_resource_map)} products loaded")
    print(f"\nSample entries:")
    for idx, (product_id, info) in enumerate(list(tier1_resource_map.items())[:5]):
        print(f"  {product_id}: {info['Resource_ID']} - {info['Resource_Description']}")

except Exception as e:
    print(f"Error loading Tier 1: {e}")

In [ ]:
print("\n" + "="*120)
print("TIER 2: LOAD L2 PLAN (L2Ph1_Detail)")
print("="*120)

tier2_resource_map = {}

try:
    df_l2_detail = pd.read_excel(l2_plan_file, sheet_name="L2Ph1_Detail", header=1)
    
    print(f"Sheet shape: {df_l2_detail.shape}")
    print(f"Columns: {list(df_l2_detail.columns[:10])}")
    
    for _, row in df_l2_detail.iterrows():
        product_id = normalize_product_no(row.get('Product ID')) if 'Product ID' in row.index else None
        product_desc = normalize_text(row.get('Product Desc')) if 'Product Desc' in row.index else None
        molecule = normalize_text(row.get('Molecule')) if 'Molecule' in row.index else None
        region = normalize_text(row.get('Region')) if 'Region' in row.index else None
        product_form = normalize_text(row.get('Product Form')) if 'Product Form' in row.index else None
        
        if product_id and product_id != '0':
            if product_id not in tier1_resource_map:
                tier2_resource_map[product_id] = {
                    'Product_Description': product_desc,
                    'Molecule': molecule,
                    'Region': region,
                    'Product_Form': product_form,
                    'Source': 'L2Ph1_Detail'
                }
    
    print(f"Tier 2 Resource Map: {len(tier2_resource_map)} additional products loaded")
    print(f"\nSample entries:")
    for idx, (product_id, info) in enumerate(list(tier2_resource_map.items())[:5]):
        print(f"  {product_id}: {info['Molecule']} - {info['Product_Form']}")

except Exception as e:
    print(f"Error loading Tier 2: {e}")

In [ ]:
print("\n" + "="*120)
print("HIERARCHY DEFINITIONS")
print("="*120)

product_mapping = {}

mCB_skus = ['800004403', '800004402', '800008019', '800008020', '800008034', '800007997', '800007345', '800007516',
            '800002513', '800007608', '800007630', '800002984', '800004986', '800007310', '800007311', '800006648',
            '800007634', '800008073', '800006523', '800002297', '800002872', '800006741', '800007380']

sMCB_skus = ['800006506', '800006505', '800006527', '800006526', '800006525', '800007546', '800007583', '800007839',
             '800006524', '800006627', '800007872']

vial_skus = ['800004400', '800004401', '800006626', '800006740', '800007996']

aspart_dlp_skus = ['800008016', '800002958', '800002948', '800006528', '800002989', '800003528', '800006592', '800006691']

aspart_vial_skus = ['800008017', '800006529']

rhi_skus = ['800001300', '800001298', '800001299']

for sku in mCB_skus:
    product_mapping[sku] = {'assembly': '700003964', 'filling': '700001012', 'root': '700001470', 'family': 'Glargine_mCB_DLP'}

for sku in sMCB_skus:
    product_mapping[sku] = {'assembly': '700004129', 'filling': '700004130', 'root': '700004130', 'family': 'Glargine_sMCB_DLP_EU'}

for sku in vial_skus:
    product_mapping[sku] = {'assembly': '700001123', 'filling': '700001123', 'root': '700001123', 'family': 'Glargine_Vial'}

for sku in aspart_dlp_skus:
    product_mapping[sku] = {'assembly': '700002770', 'filling': '700001301', 'root': '700001301', 'family': 'Aspart_DLP'}

for sku in aspart_vial_skus:
    product_mapping[sku] = {'assembly': '700001318', 'filling': '700001318', 'root': '700001318', 'family': 'Aspart_Vial'}

for sku in rhi_skus:
    product_mapping[sku] = {'assembly': '700000536', 'filling': '700000536', 'root': '700000536', 'family': 'RHI'}

print(f"Hierarchy defined: {len(product_mapping)} SKU mappings")

In [ ]:
print("\n" + "="*120)
print("RESOURCE MAPPING RULES (From Flowchart)")
print("="*120)

resource_rules = {
    'MARCHESINI': {
        'resource_id': 'MFAPL0012702001',
        'resource_desc': 'MARCHESINI Auto Pack',
        'skus': ['800002671', '800002966', '800004400', '800001298', '800001299', '800001300', '800002984', '800004986'],
        'products': ['Aspart_CA', 'Basalog', 'Vial', 'RHI']
    },
    'IMA': {
        'resource_id': 'MFIPL0012702001',
        'resource_desc': 'IMA Auto Pack',
        'skus': ['800004402', '800004403', '800002297', '800003528'],
        'products': ['Glargine_mCB_DLP', 'Basalog', 'Aspart_DLP']
    },
    'MANUAL': {
        'resource_id': 'MFMPPL012702001',
        'resource_desc': 'Manual Pack',
        'skus': ['800004400', '800004401', '800006626', '800006740', '800007996'],
        'products': ['Vial', 'Low_Volume_DLP']
    },
    'FILLING': {
        'resource_id': 'MFVFM0012702001',
        'resource_desc': 'Combi Filling Line',
        'stage': 'Filling'
    },
    'ASSEMBLY': {
        'resource_id': 'BIB03_BIB05',
        'resource_desc': 'Assembly Lines',
        'stage': 'Assembly'
    }
}

print("Resource mapping rules defined:")
for resource_type, info in resource_rules.items():
    print(f"  {resource_type}: {info.get('resource_desc', 'N/A')}")

In [ ]:
print("\n" + "="*120)
print("MERGED RESOURCE MAPPING (Tier1 + Tier2 + Rules)")
print("="*120)

consolidated_resource_map = {}

all_products = set()
for product_id in tier1_resource_map.keys():
    all_products.add(product_id)
for product_id in tier2_resource_map.keys():
    all_products.add(product_id)
for sku in product_mapping.keys():
    all_products.add(sku)
for sku_list in [v.get('assembly'), v.get('filling'), v.get('root') for v in product_mapping.values()]:
    if sku_list:
        all_products.add(sku_list)

for product_id in all_products:
    resource_info = {}
    
    if product_id in tier1_resource_map:
        resource_info.update(tier1_resource_map[product_id])
    elif product_id in tier2_resource_map:
        resource_info.update(tier2_resource_map[product_id])
    else:
        resource_info['Source'] = 'Hierarchy_Based'
    
    if product_id in product_mapping:
        hierarchy_info = product_mapping[product_id]
        resource_info['hierarchy_level'] = 'Packing'
        resource_info['assembly_id'] = hierarchy_info.get('assembly')
        resource_info['filling_id'] = hierarchy_info.get('filling')
        resource_info['family'] = hierarchy_info.get('family')
    
    consolidated_resource_map[product_id] = resource_info

print(f"Consolidated Resource Map: {len(consolidated_resource_map)} products")
print(f"\nSample mappings:")
for idx, (product_id, info) in enumerate(list(consolidated_resource_map.items())[:10]):
    resource = info.get('Resource_ID', info.get('Molecule', 'N/A'))
    source = info.get('Source', 'N/A')
    print(f"  {product_id}: {resource} (Source: {source})")

In [ ]:
print("\n" + "="*120)
print("RESOURCE LOOKUP FUNCTION")
print("="*120)

def get_resource_info(product_id, hierarchy_level='Packing'):
    """
    Get resource info for a product with two-tier lookup
    
    Args:
        product_id: Product ID to lookup
        hierarchy_level: 'Packing', 'Assembly', or 'Filling'
    
    Returns:
        dict with Resource_ID, Resource_Description, and Source
    """
    result = {
        'Product_ID': product_id,
        'Resource_ID': 'N/A',
        'Resource_Description': 'N/A',
        'Source': 'Not Found'
    }
    
    if product_id in tier1_resource_map:
        tier1_info = tier1_resource_map[product_id]
        result['Resource_ID'] = tier1_info.get('Resource_ID', 'N/A')
        result['Resource_Description'] = tier1_info.get('Resource_Description', 'N/A')
        result['Source'] = 'Tier1_DP_Line_Utilization'
    
    elif product_id in tier2_resource_map:
        tier2_info = tier2_resource_map[product_id]
        result['Product_Description'] = tier2_info.get('Product_Description', 'N/A')
        result['Molecule'] = tier2_info.get('Molecule', 'N/A')
        result['Source'] = 'Tier2_L2Ph1_Detail'
    
    elif hierarchy_level == 'Filling' and product_id == '700001012':
        result['Resource_ID'] = 'MFVFM0012702001'
        result['Resource_Description'] = 'Combi Filling Line'
        result['Source'] = 'Hierarchy_Filling_Rule'
    
    elif hierarchy_level == 'Assembly' and product_id == '700003964':
        result['Resource_ID'] = 'BIB03_BIB05'
        result['Resource_Description'] = 'Assembly Lines'
        result['Source'] = 'Hierarchy_Assembly_Rule'
    
    return result

print("Resource lookup function defined")
print("\nTest lookups:")
test_products = ['800004403', '700001012', '700003964', '700001470']
for prod_id in test_products:
    info = get_resource_info(prod_id)
    print(f"  {prod_id}: {info['Resource_ID']} ({info['Source']})")

In [ ]:
print("\n" + "="*120)
print("EXPORT RESOURCE MAPPING")
print("="*120)

mapping_export_file = "/home/supriyo/Downloads/Biocon_nw/Resource_Mapping_Master.xlsx"

mapping_data = []
for product_id, info in sorted(consolidated_resource_map.items()):
    mapping_data.append({
        'Product_ID': product_id,
        'Resource_ID': info.get('Resource_ID', 'N/A'),
        'Resource_Description': info.get('Resource_Description', 'N/A'),
        'Product_Description': info.get('Product_Description', 'N/A'),
        'Molecule': info.get('Molecule', 'N/A'),
        'Family': info.get('family', 'N/A'),
        'Hierarchy_Level': info.get('hierarchy_level', 'N/A'),
        'Assembly_ID': info.get('assembly_id', 'N/A'),
        'Filling_ID': info.get('filling_id', 'N/A'),
        'Source': info.get('Source', 'N/A')
    })

df_mapping = pd.DataFrame(mapping_data)

with pd.ExcelWriter(mapping_export_file, engine='openpyxl') as writer:
    df_mapping.to_excel(writer, sheet_name='Resource_Master', index=False)
    print(f"Exported: {mapping_export_file}")
    print(f"Total products mapped: {len(df_mapping)}")

In [ ]:
print("\n" + "="*120)
print("RESOURCE MAPPING SUMMARY")
print("="*120)

print(f"\nTier 1 (DP Line Utilization): {len(tier1_resource_map)} products")
print(f"Tier 2 (L2Ph1_Detail): {len(tier2_resource_map)} additional products")
print(f"Hierarchy Rules: {len(product_mapping)} SKU mappings")
print(f"Total Consolidated: {len(consolidated_resource_map)} products")

print(f"\nResource Types Found:")
resource_types = {}
for _, info in consolidated_resource_map.items():
    source = info.get('Source', 'Unknown')
    resource_types[source] = resource_types.get(source, 0) + 1

for source, count in sorted(resource_types.items(), key=lambda x: x[1], reverse=True):
    print(f"  {source}: {count}")

print(f"\nExport file: Resource_Mapping_Master.xlsx")